<a href="https://colab.research.google.com/github/ccosmin97/IDC-Prostate_segmentation/blob/main/prostateX_patID0004_dcmqi_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auth login

##Colab

In [1]:
#colab 
from google.colab import auth
auth.authenticate_user()

# Global variables

## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [2]:
# IDC collection for paths setup
collection_name = "prostatex" #up to user
# ##nnunet
# model_type = '3d_fullres'#other options are '2d',..
# nnunet_labelID = 2 #for task005 model, labelid=1 means PZ and 2 means TZ
# ##labelID seg retrieval
# ground_truth_cat_CodeMeaning = 'Anatomical Structure'
# ground_truth_type_CodeMeaning = 'Transition zone of prostate'
# #Resampling scheme -- this option is made for qin-rep-repeatability(SeriesDescription tag dependence,etc), not modular for other collections with multi modal inference input
# desired_grid = "T2" #Other option is "ADC"  -- resampling grid because multi-modal input for inference input 
# series_desc_resampling_pattern = "t2_tse_tra"

## Global paths

In [3]:
import os 

#data
os.environ["qin_prostate_rep_root"] = os.path.join(os.getcwd(), collection_name)
os.environ["dcm_metadata_json"] = os.path.join(os.getcwd(), "preds_task005_prostatex_metadata.json")
os.environ["qin_prostate_rep_dicom"] = os.path.join(os.environ["qin_prostate_rep_root"], "dicom")
os.environ["qin_prostate_rep_nii"] = os.path.join(os.environ["qin_prostate_rep_root"], "nii")
os.environ["qin_prostate_rep_nrrd"] = os.path.join(os.environ["qin_prostate_rep_root"], "nrrd")
os.environ["qin_prostate_rep_analysis"] = os.path.join(os.environ["qin_prostate_rep_root"], "results")
os.environ["qin_prostate_rep_analysis_data"] = os.path.join(os.environ["qin_prostate_rep_analysis"], "data_seg")
os.environ["qin_prostate_rep_analysis_verbose"] = os.path.join(os.environ["qin_prostate_rep_root"], "results_verbose")
#dicom segs
os.environ["qin_prostate_rep_data_dcm_segs"] = os.path.join(os.environ["qin_prostate_rep_root"], "data_seg_dcm")


#resampled data -- To t2 mod
os.environ["qin_prostate_rep_root_resampled"] = os.path.join(os.environ["qin_prostate_rep_root"], "resampled")
os.environ["qin_prostate_rep_nii_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nii")
os.environ["qin_prostate_rep_nrrd_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nrrd")

#misc
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")
os.environ["logs"] = os.path.join(os.getcwd(), "logs")

#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["qin", "nnunet", "IDC", "nnUNet", "logs"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Setup GCP Project ID

In [4]:
project_id = "idc-sandbox-003"
os.environ["GCP_PROJECT_ID"] = project_id

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies. 

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

## Environment Setup

Here we will configure the Linux environment variables needed to run the nnU-Net pipeline. 

Three main variables are needed by default to run the nnU-Net segmentation pipelines:
* `nnUNet_raw_data_base` is the path to the folder where the segmentation pipeline expects to find the data to process;
* `nnUNet_preprocessed` is the path to the folder where the preprocessed data are saved;
* `RESULTS_FOLDER` is the path to the folder storing by default the model weights and, in our case, for simplicity, the segmentation masks produced by the pipeline.

We will use the additional variable `PATH_TO_MODEL_FILE` to point to the location where the pre-trained model weights for the chosen model will be stored (more on this later).

Please notice that these variables need to be set using `os.environ[]` in Google Colab - as `!export` is not sufficient to guarantee the variables are kept from one cell to the other. For more in-depth information regarding what the nnU-Net framework uses these folders for, please visit [the dedicated nnU-Net documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md)

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [5]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [6]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [7]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz 
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [8]:
%%capture

!sudo apt install subversion

In [9]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled May 21 2022, 07:24:25 on x86_64-pc-linux-gnu


## Install Python packages

In [10]:
%%capture
!pip install pydicom
!pip install nibabel
!pip install pydicom-seg

Unpack and install model we downloaded earlier (under `PATH_TO_MODEL_FILE`). This step can take about 1-2 minutes.

Next we set up few things to help with visualization of the segmentations later.

In [11]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import pydicom
import pydicom_seg
import SimpleITK as sitk

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

In [12]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

A    pyplastimatch/__init__.py
A    pyplastimatch/pyplastimatch.py
A    pyplastimatch/utils
A    pyplastimatch/utils/__init__.py
A    pyplastimatch/utils/data.py
A    pyplastimatch/utils/eval.py
A    pyplastimatch/utils/viz.py
Checked out revision 18.


In [13]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [14]:
from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.utils import viz as viz_utils
from pyplastimatch.utils import data as data_utils

# Data selection, downloading and structuring

We want to select here the collection named qin-prostate repeatibility, and more particularly the two timepoints per patient ID for further analysis.

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

## Data selection -- BigQuery

Explanation

In [15]:
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])
selection_query = f"""
WITH
# studies that have mr volumes
mr_studies AS(
  SELECT
    dicom_all_mr.SOPInstanceUID as SOPInstanceUID, 
    dicom_all_mr.StudyInstanceUID as StudyInstanceUID,
    '' AS segPropertyTypeCodeMeaning, 
    '' AS segPropertyCategoryCodeMeaning 
  FROM
    `bigquery-public-data.idc_v9.dicom_all` as dicom_all_mr
  WHERE
    Modality = 'MR'
    AND collection_id = 'prostatex'
    AND  REGEXP_CONTAINS(SeriesDescription, r".*t2_tse_tra.*|.*ADC.*")),
-- ),

 
# studies that have segmentations for the whole prostate
seg_studies AS (
SELECT
  dicom_all_seg.SOPInstanceUID as SOPInstanceUID,
  dicom_all_seg.StudyInstanceUID as StudyInstanceUID,
  segmentations.SegmentedPropertyType.CodeMeaning as segPropertyTypeCodeMeaning,
  segmentations.SegmentedPropertyCategory.CodeMeaning as segPropertyCategoryCodeMeaning

FROM
  `bigquery-public-data.idc_v9.dicom_all` AS dicom_all_seg
JOIN
  `bigquery-public-data.idc_v9.segmentations` AS segmentations
ON
  dicom_all_seg.SOPInstanceUID = segmentations.SOPInstanceUID
WHERE
  collection_id = "prostatex"
AND 
  REGEXP_CONTAINS(segmentations.SegmentedPropertyType.CodeMeaning, r".*Transition zone of.*")
AND 
  segmentations.SegmentedPropertyCategory.CodeMeaning = 'Anatomical Structure'),

union_mr_seg AS(
  SELECT
    mr_studies.SOPInstanceUID, 
    mr_studies.segPropertyTypeCodeMeaning, 
    mr_studies.segPropertyCategoryCodeMeaning, 
  FROM 
    mr_studies
  JOIN
    seg_studies
  ON
    mr_studies.StudyInstanceUID = seg_studies.StudyInstanceUID
  UNION ALL
  SELECT
    seg_studies.SOPInstanceUID, 
    seg_studies.segPropertyTypeCodeMeaning, 
    seg_studies.segPropertyCategoryCodeMeaning, 
  FROM 
    seg_studies)

SELECT 
  DISTINCT(dc_all.SOPInstanceUID),
  dc_all.gcs_url,
  dc_all.SeriesInstanceUID, 
  dc_all.StudyInstanceUID,  
  dc_all.PatientID,
  dc_all.Modality, 
  dc_all.SeriesDescription,
  union_mr_seg.segPropertyTypeCodeMeaning,
  union_mr_seg.segPropertyCategoryCodeMeaning,
  (SELECT SeriesInstanceUID FROM UNNEST(dc_all.ReferencedSeriesSequence)) AS RefSerieUID 
FROM 
  `bigquery-public-data.idc_v9.dicom_all` as dc_all
INNER JOIN
  union_mr_seg
ON 
  dc_all.SOPInstanceUID = union_mr_seg.SOPInstanceUID"""

In [16]:
selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [17]:
selection_df.SeriesDescription.unique()

array(['t2_tse_tra', 'ep2d_diff_tra_DYNDIST_MIX_ADC', 'Segmentation',
       'ep2d_diff_tra_DYNDIST_ADC', 'diffusie-3Scan-4bval_fs_ADC',
       'ep2d_DIFF_tra_b50_500_800_1400_alle_spoelen_ADC',
       'diff tra b 50 500 800 WIP511b alle spoelen_ADC'], dtype=object)

### Filter t2 and ADC series corresponding to correct seg

In [18]:
ref_seg_vals = selection_df.RefSerieUID.dropna().values
#first retrieve t2 seriesUID that have segs
t2_seg = selection_df[(selection_df.SeriesInstanceUID.isin(ref_seg_vals)) & (selection_df.Modality == 'MR')]
#retrieve others serieUID, probably only ADCs
adc_series = selection_df[(selection_df.StudyInstanceUID.isin(t2_seg.StudyInstanceUID.dropna().values)) & (selection_df.Modality == 'MR') & (selection_df.SeriesDescription.str.contains('ADC', regex=False))]
#retrieve segs serieUID
seg_series = selection_df[selection_df.Modality == 'SEG']
#merge everything back
selection_df = selection_df[selection_df.SeriesInstanceUID.isin(list(t2_seg.SeriesInstanceUID.values)+list(adc_series.SeriesInstanceUID.values)+list(seg_series.SeriesInstanceUID.values))]

In [19]:
selection_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4082 entries, 0 to 4369
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   SOPInstanceUID                  4082 non-null   object
 1   gcs_url                         4082 non-null   object
 2   SeriesInstanceUID               4082 non-null   object
 3   StudyInstanceUID                4082 non-null   object
 4   PatientID                       4082 non-null   object
 5   Modality                        4082 non-null   object
 6   SeriesDescription               4082 non-null   object
 7   segPropertyTypeCodeMeaning      4082 non-null   object
 8   segPropertyCategoryCodeMeaning  4082 non-null   object
 9   RefSerieUID                     98 non-null     object
dtypes: object(10)
memory usage: 350.8+ KB


In [20]:
selection_df.PatientID.unique()

array(['ProstateX-0004', 'ProstateX-0007', 'ProstateX-0009',
       'ProstateX-0012', 'ProstateX-0015', 'ProstateX-0020',
       'ProstateX-0026', 'ProstateX-0029', 'ProstateX-0031',
       'ProstateX-0033', 'ProstateX-0039', 'ProstateX-0044',
       'ProstateX-0046', 'ProstateX-0054', 'ProstateX-0056',
       'ProstateX-0059', 'ProstateX-0060', 'ProstateX-0065',
       'ProstateX-0066', 'ProstateX-0069', 'ProstateX-0070',
       'ProstateX-0072', 'ProstateX-0076', 'ProstateX-0080',
       'ProstateX-0083', 'ProstateX-0084', 'ProstateX-0089',
       'ProstateX-0090', 'ProstateX-0094', 'ProstateX-0096',
       'ProstateX-0102', 'ProstateX-0104', 'ProstateX-0107',
       'ProstateX-0110', 'ProstateX-0111', 'ProstateX-0112',
       'ProstateX-0117', 'ProstateX-0118', 'ProstateX-0120',
       'ProstateX-0121', 'ProstateX-0125', 'ProstateX-0129',
       'ProstateX-0130', 'ProstateX-0134', 'ProstateX-0136',
       'ProstateX-0141', 'ProstateX-0142', 'ProstateX-0144',
       'ProstateX-0146',

## Pick a PatientID

In [21]:
# # #random PatientID selection
# patientID_select = random.sample(list(selection_df.PatientID.values), 1)#X randomly picked PatientIDs 

# static PatientID selection
patientID_select = ["ProstateX-0004"]

selection_df = selection_df[selection_df.PatientID.isin(patientID_select)]

In [22]:
selection_df.SeriesDescription.unique()

array(['t2_tse_tra', 'ep2d_diff_tra_DYNDIST_MIX_ADC', 'Segmentation'],
      dtype=object)

## Download gcs urls into dcm files and sorting  

In [23]:
# save the list of GCS URLs into a file
selection_manifest = os.path.join(os.environ["IDC_Downloads"], "idc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [24]:
# confirm the resulting manifest has as many lines as the number of rows in the
# dataframe we initialized earlier
!cat {selection_manifest} |wc

     39      39    2574


In [25]:
# let's make sure the download folder is clean, in case you ran this cell earlier
# for a different dataset
!rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}

In [26]:
# download is this simple!
%%capture

!cat {selection_manifest} | gsutil -m cp -I {os.environ["IDC_Downloads"]}

To facilitate preparation of the data, we will sort the downloaded files into the PatientID/StudyInstanceUID/SeriesInstanceUID hierarchy.

In [27]:
!python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 40/40 [00:00<00:00, 85.40it/s]
Files sorted


Move the sorted data into the right place

In [28]:
rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}* && mv {os.environ["IDC_Downloads_Sorted"]+"/*"} {os.environ["qin_prostate_rep_dicom"]}

#Pre-Processing

Dicom files are assumed to be sorted this way for further processing :

* PatientID
  * StudyUID
    * SeriesUID
      * *.dcm file(s)

## conversion methods

https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24283641

switch from plastimatch resample method to simple itk for slice spacing issue

In [29]:
def safe_sitk_read(img_list, *args, **kwargs):
  """
  Since the default function just looks at images 0 and 1 to determine slice thickness
  and the images are often not correctly alphabetically sorted, much slower
  :param img_list:
  :return:
  """
  pimg_list = [(sitk.ReadImage(x).GetOrigin(), x) for x in img_list]
  s_img_list = [path for _, path in sorted(pimg_list, key = lambda x: x[0][2])] # sort by z
  return sitk.ReadImage(s_img_list, *args, **kwargs)

In [30]:
def convert_mr_to_nii_nrrd(input_path, output_path_root, modality, target_format="nii.gz"):
  if not os.path.exists(output_path_root): 
    !mkdir -p $output_path_root
  output_path = os.path.join(output_path_root, input_path.split('/')[-1][:-4] 
                             + "_MR_" + modality
                             + "." + target_format)
  #convert to target format
  if not os.path.exists(output_path):
    dcm_img = safe_sitk_read(glob.glob(os.path.join(input_path, "**", "*.dcm"), recursive=True))
    sitk.WriteImage(dcm_img, output_path)

We will use the [segimage2itkimage tool](https://qiicr.gitbook.io/dcmqi-guide/opening/cmd_tools/seg/segimage2itkimage) from dcmqi to extract individual segments from DICOM SEG series we just downloaded into NII format, saving each segment into a separate file

In [31]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path): 
    !mkdir -p $output_path
  
  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii 

In [32]:
def convert_dcm_sorted(input_path,output_path):
  path_dicom = os.environ["qin_prostate_rep_dicom"] 
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#, recursive = True):
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    modality = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#'SEG' if pydicom.dcmread(path_serie_dcm_lst[0]).Modality == "SEG" else "MR"
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    SeriesDescription = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["SeriesDescription"].iloc[0]
    print(f"Series Description : {SeriesDescription}")
    type_serie = ["T2" if "t2" in SeriesDescription else "ADC" if "ADC" in SeriesDescription else "SEG" if "Segmentation" in SeriesDescription else 'not_found'][0]
    if modality == "SEG":
      #convert to nrrd
      convert_seg_to_nii(input_path=path_serie_dcm_lst[0],
                              output_path=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_{}".format(type_serie)))
    elif modality == "MR":
      #convert to nii
      convert_mr_to_nii_nrrd(input_path=serie_folder,
                              output_path_root=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nii.gz")
      #convert to nrrd
      convert_mr_to_nii_nrrd(input_path=serie_folder, 
                              output_path_root=os.path.join(output_path, "nrrd", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nrrd")
    else:
      print(f"type serie {modality} not supported")

## Conversion to NII/NRRD

In [33]:
!rm -rf /content/prostatex/nii/*
!rm -rf /content/prostatex/nrrd/*

In [34]:
convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
                   output_path=os.environ["qin_prostate_rep_root"])

Series Description : Segmentation
input path : /content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701/1.2.276.0.7230010.3.1.4.1070885483.15960.1599120307.702.dcm
output_path : /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Row direction: 1 -1.99011e-10 4.9619e-11
Col direction: 2.05103e-10 0.970296 -0.241922
Z direction: 2.66056e-17 0.241922 0.970296
Total frames: 76
Total frames with unique IPP: 19
Total overlapping frames: 19
Origin: [-117.913, -85.1499, 12.84]
Series Description : t2_tse_tra
Series Description : ep2d_diff_tra_DYNDIST_MIX_ADC


# Export SEGS  

In [35]:
nii_list = sorted(glob.glob(os.path.join('/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG', '*.nii.gz')))
nii1  = nii_list[0]
nii2  = nii_list[1]
nii3  = nii_list[2]
nii4  = nii_list[3]
# print(nii4)
print(nii_list)

['/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/1.nii.gz', '/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/2.nii.gz', '/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/3.nii.gz', '/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/4.nii.gz']


In [36]:
!itkimage2segimage --inputImageList {nii1},{nii2},{nii3},{nii4} \
--inputDICOMDirectory  /content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680 \
--inputMetadata  /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/meta.json \
--outputDICOM test_gt_0004.dcm

dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Loaded segmentation from /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/1.nii.gz
Loaded segmentation from /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/2.nii.gz
Loaded segmentation from /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/3.nii.gz
Loaded segmentation from /content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/4.nii.gz
Searching recursively /content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.

Test with pydicom -- works fine with OHIF viewer integration but only single segment supported

In [37]:
# import pydicom
# import pydicom_seg
# import SimpleITK as sitk

# dicom_series_paths = glob.glob('/content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680/*.dcm') # Paths to an imaging series related to the segmentation
# source_images = [
#     pydicom.dcmread(x, stop_before_pixels=True)
#     for x in dicom_series_paths
# ]

# segmentations = [
#     sitk.ReadImage(x, sitk.sitkUInt16)
#     for x in nii_list 
# ]

# template = pydicom_seg.template.from_dcmqi_metainfo('/content/prostatex/nii/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701_SEG/meta.json')#pydicom_seg.template.from_dcmqi_metainfo('metainfo.json')
# writer = pydicom_seg.MultiClassWriter(
#     template=template,
#     inplane_cropping=False,  # C
#     # rop image slices to the minimum bounding box on
#                              # x and y axes. Maybe not supported by other frameworks.
#     skip_empty_slices=True,  # Don't encode slices with only zeros
#     skip_missing_segment=False,  # If a segment definition is missing in the
#                                  # template, then raise an error instead of
#                                  # skipping it.
# )
# dcm = writer.write(segmentations[0], source_images)
# dcm.save_as('segmentation_simpleitk.dcm')

# EXPORT TO OHIF 

## User input 

Set these values. 

In [38]:
project_name = 'idc-sandbox-003'
location_id = 'us-central1'
my_bucket = 'nnunet_seg_test01_bucket'
my_dataset = 'nnunet_seg_test01'
my_datastore = 'nnunet_seg_test01_datastore'

Do not change the values below. 

Set up authorization.

## Package setup

Use these instructions to first deploy your own OHIF instance: 

https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#heading=h.1phazem22o0 

Create a structured report for each series:

# GCP Healthcare steps

In order to view the MR data along with the SEGS, we need to store our data in the cloud. We first need to create a dataset, and a DICOM datastore, which will hold our MR and SEG data. To do this we will first store our data in a bucket, and then copy over the contents to the DICOM datastore. 

We first need to create a dataset if it doesn't already exist. 

In [39]:
# First let's list the datasets that we already have for our particular project_id and location_id
datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))

datasets that exist for project_id idc-sandbox-003, location us-central1: ['nnunet_seg_test01']


Create a dicom store if it doesn't already exist

In [40]:
# # First list the datastores that exist in the dataset
datastores = !gcloud healthcare dicom-stores list --project $project_name --dataset $my_dataset #--format="value(ID)"
print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(my_dataset) + ': ' + str(datastores))

datastores that exist for project_id idc-sandbox-003, location us-central1, dataset nnunet_seg_test01: ['ID                           LOCATION     TOPIC', 'nnunet_seg_test01_datastore  us-central1']


Create a bucket to store the MR data and the SEG. 

In [41]:
#delete temp bucket
!gsutil rm -r gs://nnunet_mr_test01_bucket
# Create bucket to store SEG PRED -- TEST 
!gsutil mb -p $project_name gs://nnunet_mr_test01_bucket

Removing gs://nnunet_mr_test01_bucket/1.2.276.0.7230010.3.1.4.1070885483.15960.1599120307.702.dcm#1660324233326557...
Removing gs://nnunet_mr_test01_bucket/1.3.6.1.4.1.14519.5.2.1.7311.5101.105076858500192732229022944010.dcm#1660324214326345...
Removing gs://nnunet_mr_test01_bucket/1.3.6.1.4.1.14519.5.2.1.7311.5101.105945183098328980140457962163.dcm#1660324215328094...
Removing gs://nnunet_mr_test01_bucket/1.3.6.1.4.1.14519.5.2.1.7311.5101.110450510099441729742477232668.dcm#1660324216241736...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://nnunet_mr_test01_bucket/1.3.6.1.4.1.14519.5.2.1.7311.5101.120182904570469180898021983365.dcm#1660324223657681...
Removing gs://nnunet_mr_test01_bucket/1.3

Upload the CT data and the corresponding SR to a bucket

In [42]:
# Add MRI T2 and SEG to bucket 

# MRI T2 
!gsutil -m cp -r '/content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680/*' gs://nnunet_mr_test01_bucket
!gsutil -m cp -r '/content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.278358228783511961204087191158/*' gs://nnunet_mr_test01_bucket

# SEG - GT - DCMQI NII TO DCM
!gsutil -m cp test_gt_0004.dcm gs://nnunet_mr_test01_bucket

# SEG - PREDS - IDC DCM
!gsutil -m cp /content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.2.276.0.7230010.3.1.3.1070885483.15960.1599120307.701/1.2.276.0.7230010.3.1.4.1070885483.15960.1599120307.702.dcm gs://nnunet_mr_test01_bucket


Copying file:///content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680/1.3.6.1.4.1.14519.5.2.1.7311.5101.261549691085116343248768885489.dcm [Content-Type=application/dicom]...
Copying file:///content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680/1.3.6.1.4.1.14519.5.2.1.7311.5101.105076858500192732229022944010.dcm [Content-Type=application/dicom]...
Copying file:///content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.1.14519.5.2.1.7311.5101.206828891270520544417996275680/1.3.6.1.4.1.14519.5.2.1.7311.5101.284661221440591841105464105673.dcm [Content-Type=application/dicom]...
Copying file:///content/prostatex/dicom/ProstateX-0004/1.3.6.1.4.1.14519.5.2.1.7311.5101.170561193612723093192571245493/1.3.6.1.4.

Copy the MR data and SEG from bucket to the dicom datastore we created above. 

In [43]:
# Add MRI and SR from bucket to dicomstore 

!gcloud healthcare dicom-stores import gcs $my_datastore \
                                            --dataset=$my_dataset \
                                            --location=$location_id \
                                            --project=$project_name \
                                            --gcs-uri=gs://nnunet_mr_test01_bucket/**.dcm

Request issued for: [nnunet_seg_test01_datastore]
name: projects/idc-sandbox-003/locations/us-central1/datasets/nnunet_seg_test01/dicomStores/nnunet_seg_test01_datastore


Navigate to the OHIF app you created and open the DICOM datastore. 